In [ ]:
import pathlib

from astropy.convolution import Gaussian2DKernel, convolve
import astropy.coordinates as coord
from astropy.io import ascii
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from scipy.stats import binned_statistic, binned_statistic_2d

# gala
import gala.coordinates as gc
import gala.dynamics as gd
import gala.potential as gp
from gala.units import galactic

from pyia import GaiaData

In [ ]:
joined = at.Table.read(
    pathlib.Path('~/data/Asteroseismology/TESS_Hon2021_GaiaEDR3.fits').expanduser(),
    hdu=1)

edr3_actions = at.Table.read(
    pathlib.Path('~/data/GaiaEDR3/edr3-rv-good-plx-result-actions.fits').expanduser())
joined = at.join(joined, edr3_actions, 
                 keys='source_id', join_type='left')

In [ ]:
vcirc = 229 * u.km/u.s
galcen_frame = coord.Galactocentric()
galcen_frame

In [ ]:
g = GaiaData(joined)
g = g[(g.massflag == 1) & 
      (g.parallax_over_error > 5)]
c = g.get_skycoord()

In [ ]:
# MG = g.get_G0() - g.distmod
# BPRP = g.get_BP0() - g.get_RP0()

MG = g.phot_g_mean_mag - g.distmod
BPRP = g.phot_bp_mean_mag - g.phot_rp_mean_mag

In [ ]:
lims = [
    (0.5, 1.4),
    (1.4, 2.0),
    (2, 2.4),
    (2.4, 10.)
]

fig, axes = plt.subplots(1, len(lims), 
                         figsize=(5 * len(lims), 5), 
                         sharex=True, sharey=True)

for ax, lim in zip(axes, lims):
    mask = ((g.mass.value > lim[0]) & 
            (g.mass.value < lim[1]))
#     ax.plot(BPRP[mask], MG[mask], 
#             marker='o', ls='none', 
#             mew=0, ms=2., alpha=0.5)
    ax.hist2d(
        BPRP[mask].value, MG[mask].value, 
        bins=(np.linspace(0, 3.5, 128),
              np.linspace(-5, 10, 128)),
        norm=mpl.colors.LogNorm()
    )
    
ax.set_xlim(0, 3.5)
ax.set_ylim(10, -6)

In [ ]:
lims = [
    (2, 2.4),
    (2.4, 10.)
]

fig, axes = plt.subplots(1, len(lims), 
                         figsize=(5 * len(lims), 5), 
                         sharex=True, sharey=True,
                         constrained_layout=True)

bg_mask = ((g.mass.value > 0.5) & 
           (g.mass.value < 2.))

xlim = (0.5, 2.5)
ylim = (-2, 3)

for ax, lim in zip(axes, lims):
    mask = ((g.mass.value > lim[0]) & 
            (g.mass.value < lim[1]))

    ax.hist2d(
        BPRP[bg_mask].value, MG[bg_mask].value, 
        bins=(np.linspace(*xlim, 128),
              np.linspace(*ylim, 128)),
        norm=mpl.colors.LogNorm(),
        cmap='Greys'
    )
    
    ax.plot(BPRP[mask], MG[mask], 
            marker='o', ls='none', 
            color='tab:red',
            mew=0, ms=3, alpha=0.1,
            label=f'${lim[0]:.1f} < m < {lim[1]:.1f}$')
    ax.legend(loc='upper left', fontsize=14)
    
    ax.set_xlabel(r'$G_{\rm BP}-G_{\rm RP}$')

axes[0].set_ylabel('$M_G$')

ax.set_xlim(xlim)
ax.set_ylim(ylim[::-1])

fig.suptitle('Background: log density of $0.5 < m < 2$', fontsize=22)